<h1>Web-scrape Jobs details from Wuzzuf</h1>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.parse

<h3>To Format the url based on the query and the page number:</h3>

In [2]:
def generate_url(query, page_number):
    params = {'a': 'hpd', 'q': query, 'start': page_number}
    return "https://wuzzuf.net/search/jobs/?{}".format(urllib.parse.urlencode(params))

<h3>Extract the country from the location array: </h3>

In [3]:
def extract_country(location, country):
    for i in range(0, len(location)):
        # extrat the country from the location
        coun = location[i].split(',')[-1]
        country.append(coun)
        # to remove the country from the location
        location[i] = location[i].partition(coun)[0]
        # to remove the "," at the last of the location
        location[i] = location[i][:len(location[i])-1]

<h3>To clean the data: </h3>

In [4]:
def clean_data(data):
    data.strip()
    start = 0
    end = len(data) -1 
    while not data[start].isalpha():
        start += 1
    while not data[end].isalpha():
        end -= 1
    if len(data[start: end+1]) == 0:
        return "N/A"
    return data[start: end+1]

<h3>The Web-Srape function:</h3>

In [5]:
def web_scrap(query):
    job_title = [] # css-o171kl
    company = [] # css-17s97q8
    location = [] # css-5wys0k
    type_of_employment = [] # css-n2jc4m
    description = [] # css-o171kl
    country = []
    page = 0
    while True:
        response = requests.get(generate_url(query, page))

        soup = BeautifulSoup(response.content, "html5lib")

        #Store all the jobs in the jobs array
        jobs = []
        for s in soup.find_all(class_="css-1gatmva e1v1l3u10"):
            jobs += s

        if len(jobs) == 0:
            break

        for i in range(1, len(jobs)):
            # find the job title
            title_desc = jobs[i].find_all(class_="css-o171kl")

            #find the description
            desc = ""
            for j in range(0, len(title_desc)):
                # the first one is the job title.
                if j == 0: 
                    job_title.append(clean_data(title_desc[j].text))
                # if this is the last description for the job don't pt ","
                elif j == len(title_desc)-1:
                    desc += clean_data(title_desc[j].text)
                # to put the description and ","
                else:
                    desc += clean_data(title_desc[j].text)
                    desc += ", "

            # append the description
            description.append(clean_data(desc))

            # find all the details
            comp = jobs[i].find_all(class_="css-17s97q8")
            loc = jobs[i].find_all(class_="css-5wys0k")
            type_emp = jobs[i].find_all(class_="css-n2jc4m")

            # find all employment types
            all_types = ""
            for j in range(0, len(type_emp)):
                if j == len(type_emp)-1:
                    all_types += type_emp[j].span.text
                else:
                    all_types += type_emp[j].span.text
                    all_types += ", "

            # append all the details
            company.append(clean_data(comp[0].text))
            location.append(clean_data(loc[0].text))
            type_of_employment.append(clean_data(all_types))

        page += 1
    extract_country(location, country)
    dic = {
        "Job Title": job_title,
        "Company Name: ": company,
        "Location": location,
        "Country":country,
        "Employment Type":type_of_employment,
        "Description":description
    }
    return dic

In [6]:
#examples of jobs
dic_machine_learning = web_scrap("machine learning")
dic_data_analysis = web_scrap("data analysis")
dic_software_testing = web_scrap("software testing")

In [7]:
df_machine_learning = pd.DataFrame(dic_machine_learning)
df_machine_learning.head()

,Job Title,Company Name:,Location,Country,Employment Type,Description
0,Senior/Mid Senior Deep Learning Engineer,Confidential,"Mokattam, Cairo",Egypt,Full Time,"Experienced, IT/Software Development, Engineer..."
1,ML Engineer - Recommendation System,TensorGraph,"Maadi, Cairo",Egypt,"Part Time, Freelance / Project","Entry Level, IT/Software Development, Engineer..."
2,Internship - Graduates,Trufla,"Heliopolis, Cairo",Egypt,Internship,"Entry Level, IT/Software Development, Engineer..."
3,Data Scientist,Confidential,Riyadh,Saudi Arabia,Full Time,"Experienced, IT/Software Development, Analyst/..."
4,Senior Data Scientist,BBI-Consultancy,"Nasr City, Cairo",Egypt,Full Time,"Experienced, IT/Software Development, Analyst/..."


In [8]:
df_machine_learning.shape

(41, 6)

In [9]:
df_data_analysis = pd.DataFrame(dic_data_analysis)
df_data_analysis.head()

,Job Title,Company Name:,Location,Country,Employment Type,Description
0,Data Analysis Instructor (Excel - Power BI,EpsilonAI,"Nasr City, Cairo",Egypt,"Full Time, Part Time, Freelance / Project","Entry Level, IT/Software Development, Analyst/..."
1,Senior Data Analyst - Cairo,Confidential,Cairo,Egypt,Full Time,"Experienced, IT/Software Development, Analyst/..."
2,Data Analyst,Hands of Hope Physical Therapy & Wellness,"Maadi, Cairo",Egypt,Full Time,"Experienced, IT/Software Development, Analyst/..."
3,Data Analyst,Gila Electric,"New Cairo, Cairo",Egypt,Full Time,"Experienced, Accounting/Finance, IT/Software D..."
4,Senior Data Scientist,Fixed Solutions,"Sheraton, Cairo",Egypt,Full Time,"Experienced, IT/Software Development, Engineer..."


In [10]:
df_data_analysis.shape

(691, 6)

In [11]:
df_software_testing = pd.DataFrame(dic_software_testing)
df_software_testing.head()

,Job Title,Company Name:,Location,Country,Employment Type,Description
0,Software Testing Engineer,realme,"New Cairo, Cairo",Egypt,Full Time,"Entry Level, IT/Software Development, Quality,..."
1,Senior Software Test Engineer,Glamera,"Nasr City, Cairo",Egypt,Full Time,"Experienced, IT/Software Development, Quality,..."
2,Software Testing Engineer,DMS,"Heliopolis, Cairo",Egypt,Full Time,"Entry Level, IT/Software Development, Quality,..."
3,Trevipay| Senior Software Testing Engineer - E...,Kalasko,"Hurghada, Red Sea",Egypt,Full Time,"Experienced, IT/Software Development, Quality,..."
4,Software Quality Testing Engineer,Anspire Agency,"New Cairo, Cairo",Egypt,Full Time,"Entry Level, IT/Software Development, Quality,..."


In [12]:
df_software_testing.shape

(898, 6)